In [121]:
from pyhdf.SD import SD, SDC
import numpy as np
import matplotlib.pyplot as plt
import pprint
from os import listdir
import time
import h5py
import pandas as pd
from datetime import datetime, date, timedelta
import os
import sys

In [8]:
%matplotlib auto

Using matplotlib backend: TkAgg


In [107]:
def write_file(n_array, starttime, endtime):
    fpath = "D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/reduced"
    os.chdir(fpath)
    year = starttime.strftime('%Y')
    path = '-'.join((starttime.strftime('%m%d'), endtime.strftime('%m%d')))
    if not os.path.exists(year):#this is the first file being created for that time
        os.makedirs(year)
        #set index to 0

    path = '/'.join((year,path))
    path = '.'.join((path,"h5"))
    print path
    with h5py.File(str(path).replace("pkl","h5"), mode="w") as f:
        f.create_dataset("data", data=n_array.to_records(index=False))

In [9]:
file_name = 'D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/2018/MYD05_L2.A2018240.0610.061.2018240231140.hdf'
file = SD(file_name, SDC.READ)

In [10]:
datasets_dic = file.datasets()

for idx,sds in enumerate(datasets_dic.keys()):
    print idx,sds

0 Solar_Azimuth
1 Sensor_Zenith
2 Scan_Start_Time
3 Longitude
4 Quality_Assurance_Infrared
5 Solar_Zenith
6 Water_Vapor_Near_Infrared
7 Cloud_Mask_QA
8 Water_Vapor_Infrared
9 Latitude
10 Sensor_Azimuth
11 Quality_Assurance_Near_Infrared
12 Water_Vapor_Correction_Factors


In [11]:
lat_obj = file.select('Latitude') # select sds
lat = lat_obj.get()

lon_obj = file.select('Longitude') # select sds
lon = lon_obj.get()

data_obj = file.select('Water_Vapor_Infrared') # select sds
data = data_obj.get()/100.

starttime_obj = file.select('Scan_Start_Time') # select sds
starttime = starttime_obj.get()

quality_obj = file.select('Quality_Assurance_Near_Infrared')
quality = quality_obj.get()

cloud_mask_obj = file.select('Cloud_Mask_QA')
cloud_mask = cloud_mask_obj.get()

correction_obj = file.select('Water_Vapor_Correction_Factors')
correction = correction_obj.get()

In [12]:
pprint.pprint( quality_obj.attributes() )

{'Cell_Across_Swath_Sampling': [1, 1354, 1],
 'Cell_Along_Swath_Sampling': [1, 2030, 1],
 'Geolocation_Pointer': 'External MODIS geolocation product',
 'Parameter_Type': 'Output',
 '_FillValue': 0,
 'add_offset': 0.0,
 'description': 'See MODIS atmosphere QA plan for details',
 'long_name': 'Run time QA flags',
 'scale_factor': 1.0,
 'unit': 'none',
 'valid_range': [0, -1]}


In [13]:
#plt.figure()
#plt.imshow(data, cmap='jet', origin='lower', aspect='auto')
#plt.colorbar(label = 'Signal')

In [14]:
lonmin = lon.min()
lonmax = lon.max()
latmin = lat.min()
latmax = lat.max()
print lonmin, lonmax
print latmin, latmax

-81.06737 -10.923797
55.093838 78.493416


In [16]:
delta = 0.1
rows, cols = np.where((abs(lat - 72.) <= delta) & (abs(lon - -38) <= delta))

In [17]:
latsub = lat[rows][:,cols]
lonsub = lon[rows][:,cols]
datasub = data[rows][:,cols]

In [18]:
plt.figure()
plt.pcolormesh(lon, lat, data, cmap='jet')
plt.colorbar(label = 'PWV (%s)' % 'cm')
plt.clim(0.,datasub.max())
#plt.axis([lonmin, lonmax, latmin, latmax])
plt.ylabel('latitude (deg)')
plt.xlabel('longitude (deg)')
#plt.title('%s %s data binned to azimuth and revolution #' % (name, var))
plt.grid()
plt.show()

In [83]:
latstepdeg = 1.
longsetdeg = 1.
startday = 1
scalefactor = 100.

In [20]:
summit_lon = -38.4592
summit_lat = 72.5796

In [129]:
y = 2018
directory = 'D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/%d/' % y
files = listdir(directory)

pwv = []
TIME = []
quality_assurance = []
isummit_shape = []
numfiles = 0
cloudy_fraction = []

print 'total number of files: %d' % len(files)
t1 = time.time()
for f in files:
    
    file = SD(directory+f, SDC.READ)
    lat_obj = file.select('Latitude') # select sds
    latitude = lat_obj.get()

    lon_obj = file.select('Longitude') # select sds
    longitude = lon_obj.get()
    
    delta = 0.05
    rows, cols = np.where((abs(lat - summit_lat) <= delta) & (abs(lon - summit_lon) <= delta))

    if len(rows) == 0 or len(cols) == 0:
        continue
    else:
        numfiles += 1
        #print '# files: %d, ' % numfiles, f

    data_obj = file.select('Water_Vapor_Infrared') # select sds
    data = data_obj.get()/100.

    starttime_obj = file.select('Scan_Start_Time') # select sds
    starttime = starttime_obj.get()

    quality_obj = file.select('Quality_Assurance_Near_Infrared')
    quality = quality_obj.get()
    
    datasub = data[rows][:,cols]
    timesub = starttime[rows][:,cols]
    qualitysub = quality[rows][:,cols]
    
    
    pwv.append(datasub[datasub>=0.].mean())
    TIME.append(timesub.mean())
    #quality_assurance.append(qualitysub)
    clear = float(len(datasub[datasub>=0]))
    cloudy = float(len(datasub[datasub<0]))
    
    if clear == 0:
        cloudy_fraction.append(1.)
    else:
        cloudy_fraction.append(cloudy/(clear+cloudy))
        
    sys.stdout.write("\rPercent complete: %.2f, time elapsed: %.2f min" % ((float(numfiles) * 100. / float(len(files))), (time.time()-t1)/60.))
    sys.stdout.flush()
        
t2 = time.time()
print t2-t1
pwv = np.asarray(pwv)
TIME = np.asarray(TIME)
cloudy_fraction = np.asarray(cloudy_fraction)
#quality_assurance = np.asarray(quality_assurance)

total number of files: 1942
Percent complete: 0.05, time elapsed: 0.02 min

C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: Mean of empty slice.


Percent complete: 100.00, time elapsed: 46.49 min2789.51200008


In [146]:
plt.figure()
plt.plot(TIME, cloudy_fraction)

In [132]:
doysecoffset= 23*60*60*24*365 + 7*60*60*24 - 17*60*60 - 60 - 5.761243
tt = TIME + doysecoffset

In [133]:
h5data = pd.DataFrame({"pwv" : pwv})
h5data["cloudy_fraction"] = cloudy_fraction
h5data["time"] = tt 

In [134]:
starttime = datetime.fromtimestamp(tt[0])
endtime = datetime.fromtimestamp(tt[-1])
print starttime
print endtime

2018-06-01 00:20:00
2018-08-31 23:05:00.741258


In [135]:
write_file(h5data, starttime, endtime)

2018/0601-0831.h5


In [147]:
fpath = "D:/software_git_repos/polaris/polaris_data/weather_data/MODIS/reduced/2018/"
files = listdir(fpath)

In [148]:
f = h5py.File(fpath + files[1], 'r')

In [149]:
list(f.keys())

[u'data']

In [150]:
dset = f['data']

In [152]:
cloudy = dset['cloudy_fraction']
t = dset['time']

In [157]:
cloudmean = []
tcloudmean = []
i0=0
for h in range(len(t)):
    if t[h] - t[i0] >= 3600*24:
        cloudmean.append(cloudy[i0:h+1].mean())
        tcloudmean.append(t[i0:h+1].mean())
        i0=h+1
    if i0<len(t):
        cloudmean.append(cloudy[i0:len(cloudy)+1].mean())
        tcloudmean.append(t[i0:len(t)+1].mean())
        
cloudmean = np.asarray(cloudmean)#[np.logical_not(np.isnan(cloudmean))]
tcloudmean = np.asarray(tcloudmean)#[np.logical_not(np.isnan(cloudmean))]

In [162]:
scale = 1.
bins=50
values, base = np.histogram(cloudmean, bins=bins, density=True)
cumulative = np.cumsum(values)
cumu = cumulative/cumulative.max()
cumu_I = 1. - cumu

plt.plot(base[:-1]*scale, cumu_I*100, linewidth = 2)